In [68]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
from IMPORTS import *

In [69]:
Embedding_name="word2vec"
Dataset="ATE"
START_TAG = "<START>"
STOP_TAG = "<STOP>"


In [70]:
seed_value = 18
np.random.seed(seed_value)
torch.manual_seed(seed_value)

In [71]:
word2vec = KeyedVectors.load_word2vec_format('WordEmbeddings/Word2Vec.bin',binary=True)

In [72]:
train = None
test = None
val = None
with open(f"Data\processed\{Dataset}_train.json") as f1:
    train = json.load(f1)

with open(f"Data\processed\{Dataset}_test.json") as f2:
    test = json.load(f2)

with open(f"Data\processed\{Dataset}_val.json") as f3:
    val = json.load(f3)



In [73]:
X_train,Y_train = create_train_test_val(train)
X_test,Y_test = create_train_test_val(test)
X_val,Y_val = create_train_test_val(val)

In [74]:
X_train_tokenized, toke = TokenCreator(X_train,1)
X_test_tokenized = TokenCreator(X_test,0,tokenizer = toke)
X_val_tokenized = TokenCreator(X_val,0,tokenizer = toke)

Y_train_tokenized,toke2 = TokenCreator(Y_train,0)
Y_test_tokenized = TokenCreator(Y_test,0,tokenizer = toke2)
Y_val_tokenized = TokenCreator(Y_val,0,tokenizer = toke2)



In [75]:
X_train_tokenized=[[i-1 for i in j] for j in X_train_tokenized ]
X_test_tokenized=[[i-1 for i in j] for j in X_test_tokenized ]
X_val_tokenized=[[i-1 for i in j] for j in X_val_tokenized ]

Y_train_tokenized = [[i-1 for i in j] for j in Y_train_tokenized ]
Y_test_tokenized = [[i-1 for i in j] for j in Y_test_tokenized ]
Y_val_tokenized = [[i-1 for i in j] for j in Y_val_tokenized ]


In [76]:
print(Y_train_tokenized)
print(X_train_tokenized)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 2, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 2, 0, 0, 1, 2, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 

In [77]:
st1 = find_vocab(X_train_tokenized)
vocab_size  = len(st1) + 1 
vocab_size

2406

In [78]:
max_len=0

for i in X_train_tokenized:
    max_len = max(max_len,len(i))

max_len


78

In [79]:
weights = np.zeros((vocab_size, 300))
mapping = toke.word_index 
for word, index in mapping.items():
    if word in word2vec:
        weights[index:] = word2vec[word]

In [80]:
word_to_ix=toke.word_index
word_to_ix={k:v-1 for k,v in word_to_ix.items()}

In [81]:
print(toke.word_index)
print(toke2.word_index)
print(len(toke2.word_index))

{'#UK': 1, 'the': 2, '.': 3, ',': 4, 'and': 5, 'it': 6, 'to': 7, 'i': 8, 'is': 9, 'a': 10, 'of': 11, 'for': 12, 'with': 13, 'that': 14, 'my': 15, 'you': 16, 'this': 17, 'was': 18, 'in': 19, 'not': 20, 'on': 21, 'very': 22, 'but': 23, 'computer': 24, 'use': 25, 'have': 26, 'great': 27, 'they': 28, 'laptop': 29, "n't": 30, 'so': 31, 'has': 32, 'are': 33, 'battery': 34, 'had': 35, 'screen': 36, ')': 37, 'be': 38, 'as': 39, 'all': 40, '!': 41, '(': 42, "'s": 43, 'also': 44, 'me': 45, 'just': 46, 'easy': 47, 'when': 48, 'at': 49, 'or': 50, 'only': 51, 'which': 52, 'up': 53, 'life': 54, 'can': 55, 'windows': 56, 'than': 57, 'out': 58, 'if': 59, 'do': 60, 'after': 61, 'more': 62, 'keyboard': 63, 'would': 64, 'drive': 65, 'hard': 66, 'price': 67, 'from': 68, 'well': 69, 'because': 70, 'good': 71, 'one': 72, 'an': 73, 'like': 74, 'mac': 75, 'about': 76, 'quality': 77, 'its': 78, 'love': 79, 'get': 80, 'no': 81, 'software': 82, 'time': 83, 'even': 84, 'macbook': 85, 'will': 86, 'system': 87, 'ap

In [82]:
tag_to_ix = toke2.word_index
tag_to_ix[START_TAG] = len(toke2.word_index)-1
tag_to_ix[STOP_TAG] = len(toke2.word_index)
tag_to_ix={k:v-1 for k,v in tag_to_ix.items()}


In [83]:
training_data = [(i,j) for i,j in zip(X_train_tokenized,Y_train_tokenized)]
validation_data =[(i,j) for i,j in zip(X_val_tokenized,Y_val_tokenized)]
testing_data =[(i,j) for i,j in zip(X_test_tokenized,Y_test_tokenized)]


In [84]:
import sys
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()

# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,wt):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        if(wt==True):
            self.weights=torch.FloatTensor(weights)
            self.word_embeds= self.word_embeds.from_pretrained(self.weights,freeze=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))
    def _forward_alg_opt(self,feats):
        # Initialize the forward variables
        init_alphas = torch.full((1, self.tagset_size), -10000., dtype=torch.float)  # Set to a very small value
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.  # START_TAG has score 0

        # Perform computations as a single big operation
        forward_var = init_alphas
        all_emits=[]
        for feat in feats:
            # emit_score = feat.view(1, -1, 1)
            emit_score=feat.unsqueeze(1).expand(-1, self.tagset_size).unsqueeze(-1) 
            # Reshape emission score for broadcasting
            emit_score=emit_score.transpose(2,1)
            next_tag_var = (forward_var + self.transitions).unsqueeze(1) + emit_score
            all_emits.append(next_tag_var)
                # Compute next_tag_var in a single operation
            forward_var = torch.logsumexp(next_tag_var, dim=2).T 
                # Perform log-sum-exp operation along the last dimension
            
        # Add transition scores to STOP_TAG and compute the final forward variable
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha 
    
    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        # print(gold_score)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq


In [85]:
EMBEDDING_DIM = 300
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,True)
# optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)#optimization
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-4)#optimization

# Check predictions before training
with torch.no_grad():
    precheck_sent = torch.tensor(training_data[0][0], dtype=torch.long)
    precheck_tags = torch.tensor(training_data[0][1], dtype=torch.long)
    print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(10):
    print(epoch)  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()
        # optimizer.zero_grad()
        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(sentence, dtype=torch.long)
        targets = torch.tensor(tags, dtype=torch.long)
        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)
        # print(loss)
        # # Step 4. Compute the loss, gradients, and update the parameters by
        # # calling optimizer.step()
        loss.backward()
        optimizer.step()


# # Check predictions after training
# with torch.no_grad():
#     precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
#     print(model(precheck_sent))
# # We got it!

(tensor(18.6093), [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])
0


1
2
3
4
5
6
7
8
9


In [88]:
with torch.no_grad():
    pred=[]
    for i in range(len(testing_data)):
        precheck_sent = torch.tensor(testing_data[i][0], dtype=torch.long)
        pred.append(model(precheck_sent)[1])
    # precheck_sent = torch.tensor(training_data[0][0], dtype=torch.long)
    # precheck_tags = torch.tensor(training_data[0][1], dtype=torch.long)
    # pred=model(precheck_sent)
    # pred=pred[1]
    real=Y_test_tokenized


    true_labels_flat = [label for sublist in real for label in sublist]
    predicted_labels_flat = [label for sublist in pred for label in sublist]

    # Compute F1 score
    f1 = f1_score(true_labels_flat, predicted_labels_flat, average='macro')

    print("Test F1 Score:", f1)

Test F1 Score: 0.2313487115544472


In [87]:
# cnt=0
# for i in range(len(pred)):
#     if(pred[i]==real[i]):
#         print(pred[i])
#         print(real[i])
#         cnt+=1
# print(cnt)
# print(len(pred))
cnt=0
for i in range(10):
    # if(pred[i]==real[i]):
        print(pred[i])
        print(real[i])
        # cnt+=1
# print(cnt)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 2, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 2, 0, 1, 2, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 